# Least Square

In [6]:
import sys
my_path = r'/home/ilaria/Scrivania/Machine_Learning/Project_1/Project1_ML'
sys.path.insert(0,my_path + r'/code/COMMON')

import numpy as np 
import matplotlib.pyplot as plt
from proj1_helpers import * 
from implementations import *
from outliers import handle_outliers
from labels import idx_2labels
from standard import standardize
from costs import compute_loglikelihood_reg
from sigmoid import * 
from extend_features import *

In [7]:
yb, input_data, ids = load_csv_data(my_path + r'/data/train.csv', sub_sample=False)
print('Data loaded!')
print(input_data.shape)

Data loaded!
(250000, 30)


# Handle -999

In [8]:
input_data, Y = handle_outliers(input_data,yb,-999, 'mean') # substiution with mean because the standardization
                                                           #can be affected, otherwise we should delete the whole row
ind_back, ind_sig = idx_2labels(Y, [-1,1])
Y[ind_back] = 0

# get feature names 
names = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])
log = True
degree = 1
X0, features = extend_features(input_data, names, degree, log)

-999 are replaced by the mean value of the feature
---------------------------
Features have been set to the power(s): [1]
16 Features of the momentum have been added
4 logarithmic features have been added.
Data have been standardized.
---------------------------


In [9]:
X0.shape

(250000, 50)

# Features

In [11]:
idx =  [1, 13, 4, 11, 46, 0, 44, 7, 12, 42, 49, 2, 16, 10, 43, 47, 45, 22, 6, 23, 19, 48, 21, 8, 5, 9, 32, 31, 30, 33]
X = X0[:,idx]
print(X.shape)


(250000, 30)


In [14]:
Xp = build_poly(X,4)
print(Xp.shape)

Xtmp, _,_ = standardize(Xp[:,1:])
Xp[:,1:] = Xtmp 
print(Xp.shape)

(250000, 121)
(250000, 121)


# Set the model

In [15]:
initial_w = np.ones(Xp.shape[1])
max_iters = 500
gamma = 1e-1
maxiters = 2000
method = 'gd'

w, loss = least_squares_GD(Y,Xp,initial_w, maxiters, gamma)

In [18]:
y = predict_labels(w[-1],Xp)

In [19]:
y.shape

(250000,)

In [20]:
pos = 0
neg = 0
for i in range (len(Y)):
    if Y[i] == y[i]:
        pos += 1
    else:
        neg += 1
print(pos)
print(neg)

success_rate = pos/(pos+neg)
print(success_rate)

193665
56335
0.77466


# Load the data test

In [22]:
y, test_data, ids = load_csv_data(r'/home/ilaria/Scrivania/Machine_Learning/Project_1/test.csv', sub_sample=False)

print("Data loaded! Shape: ")
print(np.shape(test_data))

Data loaded! Shape: 
(568238, 30)


In [24]:
input_data, Y = handle_outliers(test_data,y,-999,'mean')

# get feature names 
names = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])
log = True
degree = 1
X0, features = extend_features(input_data, names, degree, log)
idx = [1, 13, 4, 11, 46, 0, 44, 7, 12, 42, 49, 2, 16, 10, 43, 47, 45, 22, 6, 23, 19, 48, 21, 8, 5, 9, 32, 31, 30, 33]
X = X0[:,idx]
Xp = build_poly(X,4)
Xtmp, _,_ = standardize(Xp[:,1:])
Xp[:,1:] = Xtmp 

y_pred = predict_labels(w[-1],Xp)

-999 are replaced by the mean value of the feature
---------------------------
Features have been set to the power(s): [1]
16 Features of the momentum have been added
4 logarithmic features have been added.
Data have been standardized.
---------------------------


In [25]:
y_pred.shape

(568238,)

In [26]:
y_pred[np.where(y_pred==0)] = -1

In [27]:
create_csv_submission(ids, y_pred, "boh_sub")